# Data_Cleaning_and_Processing.ipynb

在資料科學與機器學習專案中，資料前處理（Data Cleaning & Processing）是非常重要的一個步驟。在此 Notebook，我們將示範實務中常見的資料清洗與處理技巧，包括：

- 缺失值 (Missing Values) 的檢測與處理
- 異常值 (Outliers) 的檢測與處理
- 類別特徵編碼 (Categorical Encoding)
- 數值轉換 (Numerical Transformation) 如正規化、標準化

我們將透過一個模擬的範例數據集進行示範，並以 pandas 與 scikit-learn 的相關工具為主。

## 載入套件與生成範例資料

我們先行載入所需套件並生成一個模擬的資料集，以便後續示範。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 為了示範，我們建構一個虛擬資料集
np.random.seed(42)
size = 200

# 數值特徵
age = np.random.randint(18, 70, size)  # 年齡
income = np.random.normal(50000, 15000, size)  # 收入
income[np.random.choice(range(size), 5)] = np.nan  # 插入缺失值

# 類別特徵
cities = np.random.choice(['NY', 'SF', 'LA', 'missing_city'], size=size, p=[0.4, 0.3, 0.29, 0.01])
cities[np.random.choice(range(size), 3)] = np.nan  # 類別缺失值

# 二元類別特徵
gender = np.random.choice(['M', 'F'], size=size)
gender[np.random.choice(range(size), 2)] = np.nan

# 一個可能有異常值的特徵
expenses = np.random.normal(2000, 500, size)
# 加入一些離群點
expenses[np.random.choice(range(size), 3)] = [100000, 50000, 99999]

df = pd.DataFrame({
    'age': age,
    'income': income,
    'city': cities,
    'gender': gender,
    'expenses': expenses
})

df.head()

資料集已建構完畢，包含:
- `age`: 數值特徵（整數）
- `income`: 數值特徵（浮點數，有些缺失值）
- `city`: 類別特徵（含缺失值與 'missing_city' 這種特殊值）
- `gender`: 二元類別特徵（含缺失值）
- `expenses`: 數值特徵（有明顯異常值）

## 缺失值處理 (Missing Values)

首先，我們先檢查缺失值的存在情況，並展示不同的填補策略：
- **檢視缺失值分佈**
- 使用簡單統計量填補（平均值、中位數、眾數）
- 使用 KNNImputer 之類的方法進行較智慧的插補
- 若類別特徵中有特殊標記（如 'missing_city'），可考慮保留此類別或以眾數填補

實務上並無絕對最佳方法，需根據資料特性與業務需求決定。

In [ ]:
# 檢查缺失值數量
df.isnull().sum()

有缺失值的特徵：`income`, `city`, `gender`

### 簡單填補 (SimpleImputer)

對 `income`（數值）可用平均值或中位數填補；對 `gender`（類別）可用眾數填補。

In [ ]:
# 數值特徵填補: 'income' 使用中位數
# 類別特徵填補: 'city', 'gender' 使用最常出現的值(眾數)

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# 鑑別數值與類別特徵
numeric_cols = ['income', 'expenses', 'age']
categorical_cols = ['city', 'gender']

num_df = df[numeric_cols]
cat_df = df[categorical_cols]

num_filled = pd.DataFrame(num_imputer.fit_transform(num_df), columns=numeric_cols)
cat_filled = pd.DataFrame(cat_imputer.fit_transform(cat_df), columns=categorical_cols)

df_filled = pd.concat([df[['age']], num_filled[['income','expenses']], cat_filled], axis=1)
df_filled.head()

### 高級填補 (KNNImputer)

KNNImputer 根據最近鄰樣本的平均值（對數值特徵）來插補缺失值，可能比簡單統計更精細。

範例：只對數值特徵 `income` 使用 KNNImputer。

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5)
# 只對 income 作 KNN 填補，其餘已用簡單填補完成

# 我們先對原始資料中的數值特徵（age, income, expenses）嘗試 KNNImputer
num_df = df[['age','income','expenses']]
num_df_imputed = pd.DataFrame(knn_imputer.fit_transform(num_df), columns=num_df.columns)
num_df_imputed.isnull().sum()

KNNImputer 可用於數值特徵，若資料維度高則需考慮效能。

## 異常值處理 (Outlier Detection & Handling)

異常值可能來自資料錯誤錄入或真實但罕見的情境。在分析與建模前，可考慮：
- 移除異常值
- 使用 RobustScaler 等對異常值不敏感的轉換器
- 使用對異常值不敏感的模型

先透過視覺化方法（盒鬚圖、直方圖）檢視 `expenses` 特徵中離群點。

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(x=df['expenses'])
plt.title('Expenses Boxplot')
plt.show()

透過盒鬚圖可見，`expenses` 有幾個極端值（> 50000）。

處理方法範例：
1. 移除異常值（若比例很少且明顯錯誤）
2. 將異常值截斷 (Winsorization)
3. 使用 RobustScaler 對特徵進行縮放，使得離群值影響降低

範例：使用 IQR（四分位距）法則移除異常值。

In [ ]:
q1 = df['expenses'].quantile(0.25)
q3 = df['expenses'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr

mask = (df['expenses'] >= lower_bound) & (df['expenses'] <= upper_bound)
df_no_outlier = df[mask]
df_no_outlier.shape

透過該方法，我們將高於某閾值的離群值移除。

## 類別特徵編碼 (Categorical Encoding)

類別特徵（如 city, gender）需要轉換為數值表示才能餵入大多數機器學習模型。

常用方法：
- Label Encoding：將類別映射為整數標籤
- One-Hot Encoding：對類別創建二元指標欄位
- Target Encoding 等進階方法（本示範不涉）

對 `city`、`gender` 使用 One-Hot Encoding：

In [ ]:
# 範例：One-Hot Encoding
df_categorical = df_no_outlier.copy()

# 確保已無缺失值 (若有缺失值需先填補)
df_categorical['city'] = df_categorical['city'].fillna('Unknown')
df_categorical['gender'] = df_categorical['gender'].fillna('Unknown_gender')

one_hot = pd.get_dummies(df_categorical, columns=['city','gender'], drop_first=True)
one_hot.head()

上述利用 `get_dummies` 快速對類別特徵做 One-Hot。若需要在 pipeline 中使用，則可使用 `OneHotEncoder`：

In [ ]:
# 使用 sklearn OneHotEncoder 範例
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
cat_features = df_no_outlier[['city','gender']].fillna('Unknown')
encoded_array = encoder.fit_transform(cat_features)
encoded_columns = encoder.get_feature_names_out(['city','gender'])
encoded_df = pd.DataFrame(encoded_array, columns=encoded_columns)
encoded_df.head()

## 數值特徵轉換 (Scaling & Transformation)

在建模前，對數值特徵進行縮放或轉換可提高模型穩定性與訓練效率。

常見方法：
- StandardScaler：將數值特徵轉換為均值0、標準差1
- MinMaxScaler：將數值壓縮到[0,1]
- RobustScaler：對異常值不敏感，以中位數和IQR計算
- PowerTransformer：應用 Box-Cox 或 Yeo-Johnson 轉換，使特徵分佈更接近常態

範例：對 `income`、`expenses` 進行標準化與對數變換。

In [ ]:
numeric_df = df_no_outlier[['income','expenses','age']].copy()

# 使用StandardScaler
scaler = StandardScaler()
scaled_values = scaler.fit_transform(numeric_df)
scaled_df = pd.DataFrame(scaled_values, columns=numeric_df.columns)
scaled_df.describe()

可觀察 `scaled_df` 中各特徵均值接近0、標準差接近1。

### 使用 PowerTransformer

若特徵分佈偏斜可考慮 PowerTransformer（Yeo-Johnson），使數據更接近常態分佈。

In [ ]:
pt = PowerTransformer(method='yeo-johnson')
pt_values = pt.fit_transform(numeric_df)
pt_df = pd.DataFrame(pt_values, columns=numeric_df.columns)
pt_df.hist(bins=20, figsize=(12,4))
plt.suptitle('Distributions after PowerTransformer')
plt.show()

藉此，我們嘗試讓分佈更像常態，可能有助某些假設需要常態分佈的模型（如線性回歸）。

## 整合處理流程 (Pipeline)

在實務中，資料清理與特徵轉換常與模型訓練整合為一個 Pipeline，確保在訓練集與測試集上執行一致的處理步驟。

例如：
1. 對數值特徵缺失值以中位數填補
2. 對類別特徵缺失值以最頻繁值填補
3. 類別特徵 One-Hot 編碼
4. 數值特徵標準化

使用 `ColumnTransformer` 與 `Pipeline` 實現：

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

numeric_features = ['income','expenses','age']
categorical_features = ['city','gender']

numeric_transformer = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 構建一個範例 pipeline（不含模型）
preprocessed_data = preprocessor.fit_transform(df)
preprocessed_data.shape

執行後可看到產出的特徵維度增多（因 One-Hot Encoding 擴增特徵）。

有了此預處理 Pipeline，我們就能在訓練、交叉驗證與測試預測時自動化資料處理步驟。

## 小結與建議

本 Notebook 演示了常見的資料清理與處理技巧，包括缺失值填補、異常值檢測與處理、類別特徵編碼以及數值標準化與轉換。

在真實專案中，需根據資料特性、業務需求、模型選擇靈活調整策略。例如：
- 若缺失值有業務意義（如無測量值代表某狀態），可保留 'missing' 類別。
- 異常值是否移除需判斷其是否為真實有意義的 outlier。
- 類別特徵編碼方式可隨模型而定（樹模型對 One-Hot Encoding 並非一定必要）。
- 數值縮放方法（StandardScaler、MinMaxScaler、RobustScaler）視特徵分佈而定。

在後續的 `Exploratory_Data_Analysis.ipynb` 將以更深入的統計與視覺化方式探索資料分佈與特徵間關係，幫助決定進一步的特徵工程與建模策略。